#### Import the libraries

In [41]:
import os
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/MyDrive/')
# Python method used to change the current working directory to the specified path.
path = "/content/drive/MyDrive/Artigos_IC/"
os.chdir(path)
import shutil
shutil.copy('/content/drive/MyDrive/Artigos_IC/out_clean.csv', '/content/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/out_clean.csv'

In [2]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast

In [2]:
from pycaret.classification import *
from sklearn.metrics import classification_report

In [16]:
df = pd.read_csv('/content/out_clean.csv', index_col=0)
df.shape

(13489, 70)

#### Remove duplicates

In [17]:
duplicate_rows = df[df.duplicated()]
len(duplicate_rows)

2621

In [18]:
df.columns.values
num_colunas = len(df.columns)

In [19]:
print(num_colunas)

70


In [20]:
df.columns=df.columns.str.replace(' ','_')

In [21]:
df = df.drop('smell',axis=1)
df = df.drop('agreement_experts_has_smell',axis=1)
df = df.drop('severity',axis=1)
df = df.drop('type',axis=1)

## Drop Code Smell

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

In [22]:
df_smell =df.drop('agreement_experts_has_smell_name', axis=1)
df['agreement_experts_has_smell_name'] = df['agreement_experts_has_smell_name'].astype(str)
smell = df['agreement_experts_has_smell_name']
df_smell.columns

Index(['longmethod_label', 'featureenvy_label', 'dataclass_label',
       'blob_label', 'agreement_has_smell', 'ck_class_cbo',
       'ck_class_cbomodified', 'ck_class_fanin', 'ck_class_fanout',
       'ck_class_wmc', 'ck_class_dit', 'ck_class_rfc', 'ck_class_lcom',
       'ck_class_totalmethodsqty', 'ck_class_publicmethodsqty',
       'ck_class_visiblemethodsqty', 'ck_class_totalfieldsqty',
       'ck_class_privatefieldsqty', 'ck_class_finalfieldsqty', 'ck_class_loc',
       'ck_class_returnqty', 'ck_class_comparisonsqty',
       'ck_class_stringliteralsqty', 'ck_class_numbersqty',
       'ck_class_assignmentsqty', 'ck_class_variablesqty',
       'ck_class_maxnestedblocksqty', 'ck_class_uniquewordsqty',
       'ck_class_modifiers', 'ck_method_cbo', 'ck_method_wmc', 'ck_method_rfc',
       'ck_method_loc', 'ck_method_returnsqty', 'ck_method_parametersqty',
       'ck_method_methodsinvokedqty', 'ck_method_assignmentsqty',
       'ck_method_uniquewordsqty', 'stars', 'watching', 'commits'

#### Check the correlation and remove the high-correlated features from the dataset

In [23]:
smell = df['agreement_experts_has_smell_name']
corr = df.corr(numeric_only = True)
threshold = 0.8
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df.columns[columns]
high_corr = set(df.columns) - set(selected_columns)
df = df[selected_columns]
df['agreement_experts_has_smell_name']=smell
df.columns

Index(['longmethod_label', 'featureenvy_label', 'dataclass_label',
       'blob_label', 'agreement_has_smell', 'ck_class_cbo', 'ck_class_fanin',
       'ck_method_cbo', 'ck_method_wmc', 'stars', 'commits', 'loc',
       'number_of_contributors', 'cluster_kmeans_all', 'cluster_gaussian_all',
       'cluster_gaussian_loc', 'smell_organic_aglomeration',
       'smell_pmd_aglomeration', 'smell_designite_aglomeration',
       'smell_pmd_num_aglomeration', 'smell_designite_num_aglomeration',
       'smell_organic_longmethod', 'smell_organic_featureenvy',
       'smell_organic_dataclass', 'smell_organic_blob', 'smell_pmd_longmethod',
       'smell_pmd_featureenvy', 'smell_pmd_dataclass', 'smell_pmd_blob',
       'smell_designite_longmethod', 'smell_designite_featureenvy',
       'smell_designite_dataclass', 'smell_designite_blob',
       'agreement_experts_has_smell_name'],
      dtype='object')

In [24]:
df['agreement_experts_has_smell_name'].nunique()

5

In [25]:
df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True).str.strip()

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

#### Setup the model creation with pycaret

In [26]:
s = setup(data=df,
          target='agreement_experts_has_smell_name',
          feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True,
          fold=10,
          session_id=42,
          index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006834 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4105
[LightGBM] [Info] Number of data points in the train set: 32670, number of used features: 29
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

,Description,Value
0,Session id,42
1,Target,agreement_experts_has_smell_name
2,Target type,Multiclass
3,Target mapping,"blob: 0, dataclass: 1, featureenvy: 2, longmethod: 3, none: 4"
4,Original data shape,"(18799, 34)"
5,Transformed data shape,"(38310, 7)"
6,Transformed train set shape,"(32670, 7)"
7,Transformed test set shape,"(5640, 7)"
8,Numeric features,33
9,Preprocess,True


In [27]:
df_smell = get_config('X')
df_smell

,longmethod_label,featureenvy_label,dataclass_label,blob_label,agreement_has_smell,ck_class_cbo,ck_class_fanin,ck_method_cbo,ck_method_wmc,stars,...,smell_organic_dataclass,smell_organic_blob,smell_pmd_longmethod,smell_pmd_featureenvy,smell_pmd_dataclass,smell_pmd_blob,smell_designite_longmethod,smell_designite_featureenvy,smell_designite_dataclass,smell_designite_blob
0,0,1,0,0,0,10.0,2.0,8.0,3.0,970.0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,21.0,1.0,3.0,1.0,50.0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,15.0,2.0,6.0,3.0,61.0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,4.0,1.0,4.0,4.0,689.0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0.0,5.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18794,0,0,0,0,0,6.0,6.0,31.0,57.0,0.0,...,0,0,0,0,0,0,0,0,0,0
18795,0,0,0,0,0,3.0,2.0,3.0,5.0,285.0,...,0,0,0,0,0,0,0,0,0,0
18796,0,0,1,0,0,24.0,42.0,1.0,1.0,59.0,...,0,0,0,0,0,0,0,0,0,0
18797,0,0,0,1,0,16951.0,0.0,1.0,34.0,61.0,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [28]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7814,0.9224,0.7814,0.7890,0.7794,0.6783,0.6833,1.9030
rf,Random Forest Classifier,0.7795,0.9247,0.7795,0.7861,0.7765,0.6746,0.6799,2.3030
dt,Decision Tree Classifier,0.7750,0.9154,0.7750,0.7841,0.7736,0.6702,0.6751,1.1110
xgboost,Extreme Gradient Boosting,0.7631,0.9080,0.7631,0.7545,0.7563,0.6438,0.6460,1.8570
lightgbm,Light Gradient Boosting Machine,0.7580,0.9052,0.7580,0.7499,0.7522,0.6375,0.6391,8.0510
gbc,Gradient Boosting Classifier,0.6890,0.0000,0.6890,0.6872,0.6873,0.5433,0.5438,7.8210
knn,K Neighbors Classifier,0.6858,0.8687,0.6858,0.7098,0.6853,0.5558,0.5641,1.3010
ridge,Ridge Classifier,0.6229,0.0000,0.6229,0.6152,0.5708,0.4623,0.5036,1.1460
lr,Logistic Regression,0.5946,0.0000,0.5946,0.6122,0.5701,0.4214,0.4490,5.0070
nb,Naive Bayes,0.6002,0.7580,0.6002,0.5899,0.5458,0.4167,0.4598,1.1080


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

##### Result by Smell

In [29]:
# Dicionário para armazenar os relatórios por modelo e por classe
reports = {}
# Lista para armazenar os dados
data = []
for model in top5_models:
    model_name = str(model).split('(')[0]  # Obtém o nome do modelo
    predictions = predict_model(model)  # Faz previsões nos dados de teste
    #print(predictions)

    # Gerar relatório por classe
    report = classification_report(predictions['agreement_experts_has_smell_name'], predictions['prediction_label'], output_dict=True)

    # Armazenar o relatório no dicionário
    reports[model_name] = report

# Exibir o relatório para cada tipo de smell e modelo
for model, report in reports.items():
    print(f"Classification Report for {model}:")
    for smell_class in ['blob', 'dataclass', 'featureenvy', 'longmethod','none']:
        if smell_class in report:
            print(f"\nSmell: {smell_class}")
            #print(f"  Classes: {report[smell_class]}")
            print(f"  Accuracy: {report['accuracy']:.2f}")
            print(f"  Precision: {report[smell_class]['precision']:.2f}")
            print(f"  Recall: {report[smell_class]['recall']:.2f}")
            print(f"  F1-score: {report[smell_class]['f1-score']:.2f}")
            data.append({
                "Model": model,
                "Smell": smell_class,
                "Accuracy": report["accuracy"],
                "Precision": report[smell_class]["precision"],
                "Recall": report[smell_class]["recall"],
                "F1-score": report[smell_class]["f1-score"]
            })
    print("=" * 80)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8729,0.9499,0.8729,0.8679,0.8695,0.8113,0.8121


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8711,0.9495,0.8711,0.8659,0.8676,0.8086,0.8095


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.8709,0.9475,0.8709,0.8662,0.8679,0.8086,0.8093


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8649,0.9445,0.8649,0.8581,0.8600,0.7987,0.8001


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8704,0.9456,0.8704,0.8633,0.8647,0.8064,0.8083


Classification Report for ExtraTreesClassifier:

Smell: blob
  Accuracy: 0.87
  Precision: 0.65
  Recall: 0.56
  F1-score: 0.60

Smell: dataclass
  Accuracy: 0.87
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00

Smell: featureenvy
  Accuracy: 0.87
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00

Smell: longmethod
  Accuracy: 0.87
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00

Smell: none
  Accuracy: 0.87
  Precision: 0.85
  Recall: 0.90
  F1-score: 0.88
Classification Report for RandomForestClassifier:

Smell: blob
  Accuracy: 0.87
  Precision: 0.65
  Recall: 0.55
  F1-score: 0.59

Smell: dataclass
  Accuracy: 0.87
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00

Smell: featureenvy
  Accuracy: 0.87
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00

Smell: longmethod
  Accuracy: 0.87
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00

Smell: none
  Accuracy: 0.87
  Precision: 0.85
  Recall: 0.90
  F1-score: 0.87
Classification Report for DecisionTreeClassifier:

Smell: blob
  A

In [30]:
# Create DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("smell_classification_report_FS2_Multilabel_copy.csv", index=False)

In [31]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('multilabel_FS2.latex',index=False)
df3.to_csv('multilabel_FS2.csv',index=False)

## Top 10 features

In [32]:
models = ['et','rf', 'dt', 'xgboost', 'lightgbm']
top5_models = [create_model(model) for model in models]

# Plotar e salvar as imagens com nomes personalizados
for model in top5_models:
    model_name = str(model.__class__.__name__)  # Obtém o nome do modelo
    plot_model(model, plot='feature', save=True)  # Salva o gráfico

    # Renomear o arquivo salvo
    import os
    old_file = 'Feature Importance.png'  # Nome padrão gerado pelo PyCaret
    new_file = f'{model_name}_feature_importance_copy.png'  # Nome personalizado

    if os.path.exists(old_file):  # Verifica se o arquivo existe
        os.rename(old_file, new_file)  # Renomeia o arquivo

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7789,0.9181,0.7789,0.7684,0.7711,0.6681,0.6703
1,0.7956,0.9209,0.7956,0.7862,0.7878,0.6916,0.6946
2,0.7629,0.9055,0.7629,0.7603,0.7574,0.6410,0.6438
3,0.7903,0.9155,0.7903,0.7822,0.7831,0.6852,0.6878
4,0.7675,0.9171,0.7675,0.7582,0.7611,0.6517,0.6533
5,0.7865,0.9251,0.7865,0.7775,0.7793,0.6792,0.6816
6,0.6641,0.9138,0.6641,0.7997,0.6961,0.5558,0.5871
7,0.7941,0.9283,0.7941,0.7927,0.7913,0.6955,0.6967
8,0.8024,0.9309,0.8024,0.7984,0.7984,0.7054,0.7068


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7736,0.9189,0.7736,0.7620,0.7651,0.6595,0.6619
1,0.7926,0.9233,0.7926,0.7822,0.7835,0.6858,0.6897
2,0.7606,0.9093,0.7606,0.7579,0.7542,0.6361,0.6397
3,0.7850,0.9217,0.7850,0.7765,0.7769,0.6763,0.6794
4,0.7675,0.9212,0.7675,0.7584,0.7612,0.6517,0.6534
5,0.7888,0.9289,0.7888,0.7787,0.7799,0.6809,0.6845
6,0.6565,0.9140,0.6565,0.7886,0.6880,0.5451,0.5753
7,0.7971,0.9289,0.7971,0.7946,0.7930,0.6984,0.7001
8,0.8017,0.9324,0.8017,0.7969,0.7971,0.7039,0.7055


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7743,0.9119,0.7743,0.7652,0.7674,0.6620,0.6639
1,0.7933,0.9186,0.7933,0.7842,0.7859,0.6885,0.6913
2,0.7470,0.8926,0.7470,0.7421,0.7428,0.6206,0.6219
3,0.7880,0.9117,0.7880,0.7804,0.7811,0.6822,0.6845
4,0.7606,0.9087,0.7606,0.7526,0.7553,0.6426,0.6438
5,0.7842,0.9181,0.7842,0.7756,0.7771,0.6759,0.6783
6,0.6474,0.8994,0.6474,0.7937,0.6803,0.5377,0.5719
7,0.7910,0.9227,0.7910,0.7913,0.7889,0.6917,0.6929
8,0.7964,0.9252,0.7964,0.7933,0.7930,0.6971,0.6983


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7424,0.8959,0.7424,0.7298,0.7337,0.6119,0.6143
1,0.7591,0.9058,0.7591,0.7472,0.7495,0.6350,0.6387
2,0.7325,0.8957,0.7325,0.7254,0.7245,0.5927,0.5964
3,0.7682,0.9100,0.7682,0.7581,0.7599,0.6503,0.6534
4,0.7371,0.9014,0.7371,0.7277,0.7312,0.6074,0.6086
5,0.7629,0.9141,0.7629,0.7536,0.7554,0.6434,0.6459
6,0.7272,0.8882,0.7272,0.7193,0.7202,0.5875,0.5902
7,0.7652,0.9114,0.7652,0.7603,0.7619,0.6515,0.6522
8,0.7781,0.9153,0.7781,0.7727,0.7739,0.6691,0.6703


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7416,0.8944,0.7416,0.7310,0.7346,0.6128,0.6144
1,0.7523,0.9016,0.7523,0.7419,0.7449,0.6273,0.6295
2,0.7272,0.8877,0.7272,0.7209,0.7214,0.5880,0.5902
3,0.7637,0.9076,0.7637,0.7547,0.7571,0.6456,0.6475
4,0.7378,0.9013,0.7378,0.7264,0.7302,0.6064,0.6083
5,0.7584,0.9119,0.7584,0.7484,0.7516,0.6377,0.6395
6,0.7097,0.8835,0.7097,0.7024,0.7040,0.5635,0.5653
7,0.7546,0.9083,0.7546,0.7509,0.7519,0.6365,0.6371
8,0.7751,0.9128,0.7751,0.7705,0.7718,0.6657,0.6665


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

#### Ensemble the top-5 models -  não esta usando

In [33]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7789,0.9185,0.7789,0.7668,0.7699,0.6667,0.6695
1,0.7880,0.9221,0.7880,0.7775,0.7788,0.6788,0.6827
2,0.7561,0.9124,0.7561,0.7518,0.7498,0.6300,0.6331
3,0.7827,0.9226,0.7827,0.7734,0.7742,0.6726,0.6758
4,0.7713,0.9215,0.7713,0.7608,0.7636,0.6560,0.6583
5,0.7888,0.9277,0.7888,0.7793,0.7803,0.6812,0.6845
6,0.7606,0.9159,0.7606,0.7539,0.7527,0.6357,0.6399
7,0.7918,0.9289,0.7918,0.7879,0.7879,0.6904,0.6917
8,0.7971,0.9340,0.7971,0.7909,0.7919,0.6966,0.6982


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [34]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8711,0.9501,0.8711,0.8655,0.8672,0.8084,0.8094


In [35]:
result = pull()

#### Plot the ensemble model

In [36]:
result.to_csv('models_multilabel_FS2_copy.csv')

#### Finalize and save the ensemble model

In [42]:
final_best = finalize_model(blended_models)
#save_model(final_best, 'models_multilabel_FS2') or
save_model(final_best, os.path.join(path, 'models_multilabel_FS2'))

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['longmethod_label',
                                              'featureenvy_label',
                                              'dataclass_label', 'blob_label',
                                              'agreement_has_smell',
                                              'ck_class_cbo', 'ck_class_fanin',
                                              'ck_method_cbo',...
                                                               importance_type='split',
                                                               learning_rate=0.1,
                                                               max_depth=-1,
      

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.